# 🎓 Lesson 2: Image Editing & Memory Management

In this lesson, we will dive into **Image-to-Image** translation and learn how to handle heavy models on consumer hardware (managing VRAM).

### What we'll learn:
1. The Math of the `strength` parameter
2. Visualizing Latent Noise
3. Memory optimization techniques

In [ ]:
import sys
import os
from pathlib import Path
from PIL import Image
import requests
from io import BytesIO
import torch
import numpy as np

project_root = Path(os.getcwd()).parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from core.pipeline import pipeline_manager
from core.image_editor import image_editor

## 1. Load an Input Image

In [ ]:
# Download a sample image (a sketch of a cat)
url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
response = requests.get(url)
input_image = Image.open(BytesIO(response.content)).convert("RGB")
input_image = input_image.resize((512, 512))
input_image

## 2. The Science of 'Strength' (Noise Injection)

How does Image-to-Image work mathematically?

1. We encode the image to latents $z_0$.
2. We add noise to it based on `strength` ($s$).

$$z_t = \sqrt{1-s} \cdot z_0 + \sqrt{s} \cdot \epsilon$$

Let's visualize this corruption process!

In [ ]:
# Get the pipeline
pipe = pipeline_manager.get_img2img_pipeline()

# Create noise
img_arr = np.array(input_image) / 255.0
noise = np.random.normal(0, 1, img_arr.shape)

# Simulate 'strength' visually (in pixel space for demonstration)
def show_noisy_image(strength):
    # Simple linear mix for visualization
    noisy = (1 - strength) * img_arr + strength * noise
    # Clip to valid range
    noisy = np.clip(noisy, 0, 1)
    return Image.fromarray((noisy * 255).astype(np.uint8))

print("Visualizing Strength = 0.3 (Mostly original):")
display(show_noisy_image(0.3))

print("Visualizing Strength = 0.8 (Mostly noise):")
display(show_noisy_image(0.8))

## 3. Editing the Image

Now let's actually run the model. Notice how `strength=0.75` implies we start with a very noisy image, giving the model freedom to hallucinate new details.

In [ ]:
prompt = "A beautiful realistic mountain landscape, 8k, photorealistic, sunset"

# Try with strength 0.75
result, _ = image_editor.edit_image(
    image=input_image,
    prompt=prompt,
    strength=0.75,
    num_steps=30
)

result

## 4. Memory Optimization (Concepts)

Stable Diffusion requires a lot of VRAM. If you are running on a card with less than 8GB VRAM, you might encounter `OutOfMemory` errors.

Our project handles this in `core/memory_manager.py`.

In [ ]:
from config import device_config
print(f"Attention Slicing: {device_config.enable_attention_slicing}")
print(f"VAE Tiling: {device_config.enable_vae_tiling}")